<a href="https://colab.research.google.com/github/MiguelAngel9604/spark_exercice_1/blob/main/Spark_Exercise_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>